# South Atlantic BeachBirds

This indicator was used in the 2022 base blueprint.

In 2022, it was not used in all zonation areas where the source data overlaps, so we added a step to clip it down to the area where it was used in zonation. We will revisit the exent in 2023

This indicator was originally made for the South Atlantic Blueprint using zonation to combine beach bird species data.

Created by Amy Keister, last run by Amy Keister on 2 August 2022. It took 1 minute to run

In [63]:
import os
import arcpy

In [64]:
import time
start = time.time()

In [65]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBPost.gdb"

In [66]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBeachBirds.tif"

In [67]:
# define sub-indicator outputs

In [68]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [69]:
# define inputs
z= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBeachBirds2023Zonation\SABeachBirdsWarp1_RemoveInputMax0\outputs\SABeachBirds.CAZ_.rank.compressed.tif"
VH= r"F:\GIS_DATA\SpeciesAndHabitats\BeachBirds\VonHolle2011\GIS\SHOREBIRD_REGIONAL_POLY_WITH_RANK\SHOREBIRD_REGIONAL_POLY_WITH_RANK.shp"
SubR= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintSubRgn.shp"
NLCD= r"F:\GIS_DATA\LanduseLandcover\NLCD\NLCD_landcover_2019_release_all_files_20210604\nlcd_2019_land_cover_l48_20210604\nlcd_2019_land_cover_l48_20210604.img"
SAraster= r"F:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Data_Download_20220204\1_ExtentLayers\SA2021extent.tif"
State= r"F:\GIS_DATA\Boundaries\States\tl_rd22_us_state\tl_rd22_us_state.shp"

### Start analysis

In [70]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [71]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBPost.gdb


### Bring in zonation results from Brad

In [10]:
# take zonation results times 100
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent, extent=Rextent):
    out_raster = arcpy.sa.Times(z, 100); out_raster.save("birds")

In [11]:
# reclassify
out_raster = arcpy.sa.Reclassify("birds", "VALUE", "NODATA 0;0 20 1;20 40 2;40 60 3;60 80 4;80 100 5", "DATA"); out_raster.save("ind")

### Add in zeros

In [12]:
# buffer VonHolle data to make an estimate of extent
# buffer by 50 km to try to retain nearshore hardbottom
# 50 km was chosen by a quick visual exploration, this number might need to be tweaked
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer(VH, "VHbuff25", "25 Kilometers", "FULL", "ROUND", "None", None, "PLANAR")

In [13]:
# dissolve buffers (when I try to dissolve while doing the buffer it takes forever)
arcpy.management.Dissolve("VHbuff25", "VHbuff50Dis", "BUFF_DIST", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SouthAtlanticBeach\\SouthAtlanticBPost.gdb\\VHbuff50Dis'>

In [14]:
# remove all areas outside the 50km buffer we made above
out_raster = arcpy.sa.ExtractByMask("ind", "VHbuff50Dis"); out_raster.save("indic")

In [15]:
# clip to old south atlantic LCC boundary
out_raster = arcpy.sa.ExtractByMask("indic", SAraster); out_raster.save("indica")

In [16]:
# remove buffer area in Virginia, since data didn't cover VA
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.analysis.Select(State, "State", "NAME IN ('Georgia', 'Florida', 'South Carolina', 'North Carolina')")
    out_raster = arcpy.sa.ExtractByMask("indica", "State"); out_raster.save("indicat")

In [84]:
# use NLCD to remove deep marine
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con(NLCD, "indicat", "", "Value > 0"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [85]:
# clip to base Blueprint extent
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("Mask")

In [86]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("Mask", Out, '', None, "15", "NONE", "NONE", "4_BIT", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [87]:
# set code block for next step
codeblock = """
def Reclass(v):
	if v == 5:
		return '5 = >80th percentile of importance for bird index species (American oystercatcher, Wilson’s plover, least tern, piping plover) '
	elif v == 4:
		return '4 = >60th-80th percentile of importance'
	elif v == 3:
		return '3 = >40th-60th percentile of importance'
	elif v == 2:
		return '2 = >20th-40th percentile of importance'
	elif v == 1:
		return '1 = ≤20th percentile of importance for bird index species'
	elif v == 0:
		return '0 = Open water or not identified as important for bird index species'
"""

In [88]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(Out, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SouthAtlanticBeach\\SouthAtlanticBeachBirds.tif'>

In [89]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 127
	if Value == 4:
		return 201
	if Value == 3:
		return 242
	if Value == 2:
		return 253
	if Value == 1:
		return 253
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 29
	if Value == 3:
		return 109
	if Value == 2:
		return 178
	if Value == 1:
		return 220
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 19
	if Value == 3:
		return 75
	if Value == 2:
		return 123
	if Value == 1:
		return 175
	else:
		return 255
"""

In [90]:
# calculate Red field
arcpy.management.CalculateField(Out, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(Out, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(Out, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SouthAtlanticBeach\\SouthAtlanticBeachBirds.tif'>

In [24]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

915.492255449295
